# Now, run this notebook on the dataframe created by the first notebook
**<span style="color:red">Note: This notebook will process the file prepared by the previous notebook.**</span>

**Import libs and start session**

In [1]:
!pwd

/home/jovyan/work/shared/hasham/pocs/bms/new_drug_end_point/Drug_Dev_Notebooks_Data_Ext


In [2]:
import os
import json

# ! pip install spark-nlp-display
# ! pip install pickle5

with open('/home/jovyan/work/shared/hasham/jsl_keys.json', 'r') as f:
    license_keys = json.load(f)

license_keys.keys()

secret = license_keys['SECRET']
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID'] = license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
sparknlp_version = license_keys["PUBLIC_VERSION"]
jsl_version = license_keys["JSL_VERSION"]

import sparknlp
print (sparknlp.version())
import sparknlp_jsl
print (sparknlp_jsl.version())


import random
import base64
import pandas as pd
import numpy as np
import os
import json

# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

#Pyspark Imports
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F
from pyspark.sql.types import *

from datetime import date

from sparknlp_jsl.annotator import *
from sparknlp.base import *

from sparknlp.training import CoNLL
from sparknlp_jsl.training import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)


builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "45G")\
        .config("spark.driver.maxResultSize", "4096M")\
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.3") \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+secret+"/spark-nlp-jsl-3.5.1.jar")
        #.config("sparknlp.settings.pretrained.cache_folder", "/home/ubuntu/hasham/pocs/ge")

spark =  builder.getOrCreate()

print ('>> Spark Session Started')

3.4.3
3.5.1
>> Spark Session Started


In [ ]:

! pip install --upgrade spark-nlp-jsl==3.5.1 --extra-index-url https://pypi.johnsnowlabs.com/3.5.1-37f05e72855bd809ddaff47a7dee4f7728eb3cae



Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Exception in thread "main" java.nio.file.NoSuchFileException: /tmp/tmp7stee4e_/connection1537768362663260948.info
	at sun.nio.fs.UnixException.translateToIOException(UnixException.java:86)
	at sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:102)
	at sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:107)
	at sun.nio.fs.UnixFileSystemProvider.newByteChannel(UnixFileSystemProvider.java:214)
	at java.nio.file.Files.newByteChannel(Files.java:361)
	at java.nio.file.Files.createFile(Files.java:632)
	at java.nio.file.TempFileHelper.create(TempFileHelper.java:138)
	at java.nio.file.TempFileHelper.createTempFile(TempFileHelper.java:161)
	at java.nio.file.Files.createTempFile(Files.java:852)
	at org.apache.spark.api.python.PythonGatewayServer$.main(PythonGate

In [2]:
# with open('./all_endpoints_tm.txt', 'r') as f_:
#     lines = f_.readlines()
    
# dd = { "mappings": []}
# for line in lines:
#     dd['mappings'].append(
#     {
#       "key": line.strip(),
#       "relations": [
#         {
#           "key": "standard",
#           "values" : ["standard_PFA"]
#         }
#       ]
#     }
#     )
# with open('chunk_mapper_temp.json', 'w') as f_:
#     f_.write(json.dumps(dd))

**Define the pipeline**

In [3]:
from sparknlp.base import *
from sparknlp_jsl.annotator import *
sparknlp.annotator.BertForTokenClassification
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector().setCustomBounds(["\r\n","\n","\r",": ","; ","\. ", ", ", "due to", "and a"])\
    .setInputCols(["document"]) \
    .setOutputCol("sentence")
     #.setCustomBounds(["\r\n","\n","\r",": ","; ","\. "])\

tokenizer = Tokenizer()\
  .setInputCols("sentence")\
  .setOutputCol("token")

tm = TextMatcher().setInputCols("sentence","token").setOutputCol("textmatch_chunk").setEntityValue("End_Point")\
    .setEntities("./all_endpoints_tm.txt").setCaseSensitive(False).setMergeOverlapping(True)\
    .setBuildFromTokens(True)

chunkerMapper = ChunkMapperApproach()\
      .setInputCols(["textmatch_chunk"])\
      .setOutputCol("mappings")\
      .setDictionary("./chunk_mapper_temp.json")\
      .setRel("standard") #or treatment

# bert_ner = MedicalBertForTokenClassifier.pretrained("bert_token_classifier_drug_development_trials", "en", "clinical/models")\
#         .setInputCols("token", "sentence")\
#         .setOutputCol("bert_ner")\
#         .setCaseSensitive(True)

bert_ner_new = BertForTokenClassification.load("model_epoch_16_tf_spark_nlp")\
            .setInputCols("token", "sentence")\
            .setOutputCol("bert_ner")\
            .setCaseSensitive(True)

bert_ner_converter = NerConverterInternal()\
        .setInputCols(["sentence","token","bert_ner"])\
        .setOutputCol("bert_ner_chunk").setGreedyMode(True).setWhiteList(['Duration', 'Patient_Count','Hazard_Ratio',
                                                                          'Confidence_Range', 'Trial_Group'])

ner_conv_conf_level =  NerConverterInternal().setInputCols(["sentence","token","bert_ner"]).setOutputCol("chunk_conf_level").setGreedyMode(True).setWhiteList(['Confidence_level'])

emb = WordEmbeddingsModel.pretrained("embeddings_clinical","en", "clinical/models")\
    .setInputCols(['sentence', 'token'])\
    .setOutputCol("embs")

# ner = MedicalNerModel().pretrained('ner_deid_generic_augmented', 'en', 'clinical/models')\
#     .setInputCols(['token', 'embs', 'sentence']).setOutputCol('ner')

# conv = NerConverterInternal().setInputCols("sentence","token", 'ner').setOutputCol("ner_chunk").setWhiteList(['Date'])

value_matcher = RegexMatcher()\
  .setExternalRules("./value_rules.txt", ",", "TEXT")\
  .setStrategy("MATCH_ALL").setInputCols(["sentence"]).setOutputCol("value_chunk")

percent_matcher = RegexMatcher()\
  .setExternalRules("./percent_rules.txt", ",", "TEXT")\
  .setStrategy("MATCH_ALL").setInputCols(["sentence"]).setOutputCol("percent_chunk")

cmrh = ChunkMergeApproach().setInputCols("value_chunk", 'bert_ner_chunk').setOutputCol("ner_merge_chunk")\
    .setMergeOverlapping(True)

cmrh_conf_per = ChunkMergeApproach().setInputCols("chunk_conf_level", 'percent_chunk').setOutputCol("chunk_precent_conf_level")\
    .setMergeOverlapping(True)

cmrh1 = ChunkMergeApproach().setInputCols("chunk_precent_conf_level", "ner_merge_chunk").setOutputCol("all_merge_chunk")\
    .setMergeOverlapping(True)

cmrh2 = ChunkMergeApproach().setInputCols("all_merge_chunk", "textmatch_chunk").setOutputCol("rex_text_chunk")\
    .setMergeOverlapping(True)

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")
    
dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tags", "token"])\
    .setOutputCol("dependencies")

re_model = RelationExtractionModel.pretrained('posology_re', 'en', 'clinical/models')\
    .setInputCols(["embs", "pos_tags", "rex_text_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setRelationPairs(['End_Point-Value', 'Duration-End_Point'])\
    .setMaxSyntacticDistance(0)

# .setRelationPairs(['duration-end_point', 'end_point-value', 'end_point-hazard_ratio', 'trial_group-end_point'
#                       ])\

pipeline =  Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        tm,
        chunkerMapper,
        emb,
        bert_ner_new,
        bert_ner_converter,
        ner_conv_conf_level,
        emb,# ner, conv, 
        value_matcher,
        percent_matcher,
        cmrh, cmrh_conf_per, cmrh1, cmrh2,
        pos_tagger, dependency_parser, re_model
    ]
)

print (pipeline.stages)

p_model = pipeline.fit(spark.createDataFrame(pd.DataFrame({'text': ['']})))

l_model = LightPipeline(p_model)


2022-06-07 22:02:36.414043: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-07 22:02:36.759807: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2799995000 Hz


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ]embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
Download done! Loading the resource.
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ]pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
Download done! Loading the resource.
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ]dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
Download done! Loading the resource.
[ / ]

[ \ ]

[OK!]
Pipeline_6f995acc23c5__stages


In [ ]:
#10 to 14.8 months

In [4]:
%%time
from sparknlp_display import *
#txt = "In January 2007, a multicenter phase III trial (     NCT00405756  ; CC-5013-MM-015) began in 450 patients ages >/= 65 years with NDMM, aimed at comparing melphalan, prednisolone and lenalidomide to melphalan and prednisolone. At that time, completion was due in December 2012 ; the trial completed patient accrual in summer 2008 . In July 2009, the first pre-specified interim analysis of the 459-patient, international, randomized, double-blind, placebo-controlled trial was performed at 50% of the planned number of PFS events. The independent data monitoring committee reported that there was a significant improvement in the primary efficacy endpoint of PFS in the melphalan, prednisolone and lenalidomide group compared with the melphalan and prednisolone group. There were no safety concerns . In December 2009, data were presented at the 51st ASH meeting in New Orleans, LA. Patients received either lenalidomide in combination with melphalan and prednisone, followed by lenalidomide alone (MPR-R; arm 1), lenalidomide in combination with melphalan and prednisone, followed by placebo (MPR; arm 3), or placebo, melphalan and prednisone, followed by placebo (MP; arm 2). The overall response rate for patients in the MPR-R group was 77% versus 49% for patients in the MP group. Adverse events included neutropenia (70% MPR-R versus 31% MP) and thrombocytopenia (38% MPR-R versus 13% MP). The median PFS of MPR-R had not been reached, while the MP arm had a median PFS of 13 months. Patients treated with MPR-R had a 50% reduction in the risk of disease progression compared to MP , . In June 2010, further data from the 459-patient study were presented at the 15th annual meeting of the European Hematology Association in Barcelona, Spain. Patients in the MP group had a median PFS of 13 months while at that time, the median PFS for the MPR-R group had not yet been reached. Furthermore, it was reported that patients treated with MPR-R now had an improved 58% reduction in the risk of disease progression compared to MP; it was estimated that at 2-years, 55% of patients who were receiving MPR-R would be progression-free compared to 16% of patients receiving MP . In December 2010, further data were presented at the 52nd ASH meeting in Orlando, FL. At a median follow-up of 21 months, continuous lenalidomide therapy with MPR-R resulted in a higher overall response rate and higher rates of CR and very good partial responses, compared with fixed duration MP treatment (77%, 14% and 32% for MPR-R versus 50%, 4% and 12% for MP, respectively). The median PFS of the MPR-R arm was 31 months, compared with 13 months for the MP arm. Furthermore, MPR-R-treated patients demonstrated an improved 60% reduction in the risk of disease progression and a 69% reduction in the risk of disease progression compared to MP. PFS was extended in patients receiving continuous lenalidomide therapy, regardless of disease stage. At that time, follow up was too short to identify significant overall survival differences between groups. Safety analysis (all patients who received at least one dose of study therapy) showed that the most common grade IV hematological adverse events included neutropenia (36% MPR-R; 8% MP), thrombocytopenia (13% MPR-R; 4% MP) and anemia (5% and 1%, respectively). However higher rates of deep vein thrombosis (3% MPR-R; less than 1% for MP) and fatigue (6% MPR-R; 3% MP) were observed in the MPR-R arm; Grade III/IV peripheral neuropathy was experienced by 1% of patients in the MP arm and none in the MPR-R arm . In May 2011, further data were presented at the 13th International Myeloma Workshop in Paris, France. The risk of second primary malignancies, death or disease progression at 2 years for MPR-R-treated patients was 48%, versus 83% for MP patients . In December 2011, quality of life (QoL) data were presented at the 53rd ASH meeting in San Diego, CA. Statistically significant improvements in health-related QoL were seen only with MPR-R; global and physical function QoL improved and fatigue, pain, disease symptoms and treatment side-effect scores decreased. No significant changes in the HRQoL were seen with MP. HRQoL scores for physical functioning and fatigue were elevated significantly during treatment when compared with during progressive disease/treament discontinuation . Further data were presented at the same conference. In patients aged 75 years and below treated with continuous MPR-R therapy, a median PFS of 31 months was observed compared with 12 months for the MP arm. MPR-R-treated patients in this age group achieved a 70% reduction in the risk of disease progression compared with the MP arm. A trend for extended overall survival was seen with MPR-R compared with MP. A significant PFS benefit of 15 months was achieved in MPR induction alone compared with 12 months for MP; the median time to response (TTR) was 2 and 3 months for MPR and MP, respectively. In the preplanned landmark analysis, lenalidomide maintenance reduced the risk of progression by 65% for all patients irrespective of age . In May 2012, similar data were published , , . In December 2012, further data were presented from a post-hoc analysis of data from pivotal, double-blind, randomized, placebo-controlled at the 54th ASH meeting in Atlanta, GA.The percentage of patients who showed progression was 54% in arm 1 when compared with 83 and 77% of patients in arm 2 and 3, respectively. The median first-line time to progression (TTP) for arm 1 versus the overall population was 20 versus 29 months, respectively. The second-line therapy was received by 30, 58 and 72% of patients in arm 1, 2 and 3, respectively . Later that month, further mature survival data from the trial were expected in first quarter of 2013 ; in April 2013, data were presented at the International Myeloma Workshop meeting in Kyoto, Japan. The data demonstrated that event-free survival including SPM with favorable benefit-risk profile . In December 2013, similar data were presented at the 55th ASH Meeting in New Orleans, LA ."
#txt = """In August 2020, a single arm, randomized, open label, phase Ib trial ( CTR20201697 ; ALSC012AST2818 ) was planned to start in China to evaluate the ORR, DCR, DOR, DepOR, PFS, OS, and CNSORR of the drug in patients ( expected n = 30 ) with locally advanced or metastatic NSCLC with EGFR 20 exon insertion mutation; at that time, it was not yet open for recruitment . In September 2021, cohort 1 data ( n = 10 ) from the phase Ib trial ( NCT04858958; FAVOUR ) were presented at the Virtual 2021 ESMO Congress. Independent review committee ( IRC ) and investigator assessed confirmed objective response rate was 60 ( primary endpoint ) and 70%; disease control rate was 100 and 100%; complete response rate was 0 and 0%; partial response rate was 60 and 70%; stable disease rate was 40 and 30%; progressive disease rate was 0 and 0%, respectively. Tumor shrinkage in target lesions was observed in all patients with a median best percent change of – 51.8%. Median duration of exposure was 4.1 months and all the patients remained on treatment. The proportion of patients with treatment emergent adverse event ( TEAE ) , treatment related AE, serious AE, dose interruption due to an AE was 90, 80, 10 and 10%, respectively. Grade >/= 3 TEAE, dose reduction and dose continuation due to AE were not reported ."""

txt = """November 9, 2021 [Pooled Analysis]

Journal for ImmunoTherapy of Cancer, Volume 9, Issue Supplement 2, November 2021

Presented at the 36th Annual Meeting of Society for Immunotherapy of Cancer, November 10 14, 2021, Washington, DC and Virtual Meeting
Available online date: November 9, 2021
Abstract No.: 441

Benchao Chen, Xiaoping Jin, Yongcheng Dong, Max Wang, Dennis Xia, Michelle Xia and Baiyong Li

Population Pharmacokinetics of an Anti PD 1 Antibody, Penpulimab in Patients with Advanced Malignancies.

Results:
The PopPK of penpulimab was adequately described with a two compartment model with first order elimination using a non linear mixed effects model ( NONMEM ) . Typical values for these PK parameters were estimated at 0.232 L/day for Clearance ( CL ) , 3.42 L for the volume of distribution of central compartment ( V1 ) . There was no clinical correlation between penpulimab pharmacokinetics and baseline covariates, such as age, race, immunogenicity, hepatic function, creatinine clearance, tumor type, and ECOG score. Several indicators, including the impact of albumin, lactic dehydrogenase, and gender on CL, and the impact of weight and gender on V1, were found to be statistically significant, but the geometric mean ratios for the PK parameters ranged between 0.7 and 1.3, with a difference of less than 30% and no clinically significant relevance. Drug exposure increased proportionally with dose from 1.0 mg/kg to 10.0 mg/kg, and at 1mg/kg or more, the PD 1 receptor occupancy reached 80% to 100% two days after penpulimab infusion. PopPK simulation showed that concentration ranges of penpulimab following administration at a fixed dose ( 200 mg Q2W ) or by body weight ( 3.0 mg/kg Q2W ) were basically consistent. More than 97.5% of the patients, at either dosing regimen ( 200 mg Q2W or Q3W ) , had steady state trough concentrations >10 µg/mL, which is much higher than the concentration ( 0.5 µg/mL ) needed for reaching 90% to 100% PD 1 receptor occupancy.

Conclusions:
The PopPK model adequately assessed the pharmacokinetic characteristics and impact factors of penpulimab in the hematologic malignancies as well as solid tumors. This simulation indicates that both weight based dose and fixed dose ( 200mg Q2W or Q3W ) are appropriate for penpulimab.

https://jitc.bmj.com/content/9/Suppl_2/A471; May 19, 2021 [Interim results]

J Clin Oncol 39, 2021 ( suppl 15; abstr e21072 )

Presented at the 2021 ASCO Virtual Annual Meeting, June 4 8, 2021
Abstract No: e21072

Baohui Han, Jianhua Chen, Ziping Wang, Xingya Li, Lin WANG, Lin Wu, Qiang Xie, Yanqiu Zhao, Mengzhao Wang, Junguo Lu, Huaqiu Shi, Feng Ye, Xiaodong Mei, Weirong Yao, Lu Yue, Da Jiang, Jian Zhao, Shucai Zhang, Gongyan Chen, Shunchang Jiao;

Penpulimab in combination with anlotinib as first line treatment in advanced nonsquamous non small cell lung cancer

Results:
As of January 13, 2021, 26 pts had received the combination therapy of penpulimab plus anlotinib ( median age 59.5yrs [range 30 71], 76.9% male, 76.9% ECOG PS 1 ) , with a median treatment duration of 3.5 months. Of 21 pts who have had at least one tumor assessment, the ORR was 57.1% ( 12 PRs ) and DCR was 90.5% ( 12 PRs, 7 SDs ) . Median PFS has not been reached, and eleven responders remain in response. Treatment related adverse events ( TRAEs ) occurred in 53.8% of pts ( > or = G3 TRAEs occurred in 15.4% [4/26] ) . Treatment related SAEs occurred in 15.4% [4/26], and 7.7% of pts [2/26] had drug interruption or discontinuation due to TRAEs. Most common TRAEs ( > or = 15% ) were ALT increased, AST increased, hyperthyroidism and hypertension ( 15.4% each ) .

Conclusions:
The combination of penpulimab plus anlotinib as first line treatment for locally advanced/metastatic NSCLC showed the promising efficacy with a manageable safety profile, thereby suggesting that this combination therapy may be a viable chemo free treatment strategy for locally advanced/metastatic NSCLC pts.

https://meetinglibrary.asco.org/record/201091/abstract; December 3, 2020 ( Pooled analysis )

Presented at the ESMO IMMUNO ONCOLOGY CONGRESS 2020, December 9 12, 2020, Geneva, Switzerland
Available online date: December 3, 2020
Abstract No.: 31P

N. Xu, B. Han, S. Jiao, C. Hu, A. Mislang

Integrated safety analysis of anti programmed cell death 1 ( PD 1 ) antibody penpulimab in advanced solid tumour or lymphoma

Results
As of Jan 10 2020, a total of 422 patients ( pts ) [monotherapy: combo therapy= 329:93] received a median of 8 doses [1 52] of Penpulimab. Main types of tumor ( > or =10 pts ) in this analysis include: R/R cHL ( N=94 ) , NPC ( N=89 ) , NSCLC ( N=56 ) , HCC ( N=53 ) , gastric/GEJ cancer ( N=21 ) , ovarian cancer ( N=20 ) , pancreatic cancer ( N=11 ) , esophageal cancer ( N=11 ) and biliary duct carcinoma ( N = 11 ) . irAEs occurred in 23.0% of pts ( G3 in 2.6%, no G4 or G5, serious in 0.7%, treatment discontinuation in 0.7% ) . Most common irAEs ( > or =1% ) were hypothyroidism ( 9.2% ) , hyperthyroidism ( 4.3% ) , increased TSH ( 2.8% ) and rash ( 2.4% ) .

Conclusions
Penpulimab demonstrates impressive safety profile with relatively lower incidence rate of > or = Grade 3 immune related adverse events.

https://oncologypro.esmo.org/meeting resources/esmo immuno oncology virtual congress 2020/integrated safety analysis of anti programmed cell death 1 pd 1 antibody penpulimab in advanced solid tumour or lymphoma"""


# row = to_process.iloc[0]
# txt = row['paragraph']
# nct = row['nct_id']
# print (nct)
def normalize_input(txt):
    import re
    txt = txt.replace('·', '.')
    txt = txt.replace("(", " ( ").replace(")", " ) ")
    txt = txt.replace("-", " ")
    txt = txt.replace("vs.", "vs")

    def replace(g):
        return g.group(0).replace(',', ' ').replace(';', ' ').replace(':', ' ')

    txt = re.sub(r'\(.*?\)', replace, txt)
    
    return txt

res = l_model.fullAnnotate(normalize_input(txt))
NerVisualizer().display(res[0], 'rex_text_chunk')
# RelationExtractionVisualizer().display(res[0], relation_col = 'relations', #specify relations column
#                document_col = 'document', #specify document column
#                show_relations=True, exclude_relations=[])


CPU times: user 1.02 s, sys: 648 ms, total: 1.67 s
Wall time: 6.33 s


**Helper function to link entities**

In [5]:
def find_formula(k, l, mode='small', respect_section=None):
    
    section_details = None
    if respect_section != None:
        section_details = list(range(int(respect_section['begin']), int(respect_section['end']))) ## get section boundary details
    
    try:
        lrgst = min(filter(lambda x: x > k, l))
    except:
        lrgst = None
    try:
        clst = min(list(filter(lambda x: x != k, l)), key=lambda x: abs(x-k))
    except:
        clst = None
    try:
        smlst = max(filter(lambda x: x < k, l))
    except:
        smlst = None
    try:
        first = min(l)
    except:
        first = None
        
    if smlst != None and respect_section != None:
        if smlst not in section_details:
            smlst = None
    if clst != None and respect_section != None:
        if clst not in section_details:
            clst = None
    if lrgst != None and respect_section != None:
        if lrgst not in section_details:
            lrgst = None 
    
    return first, smlst, clst, lrgst


**Run pipeline and get raw results**

In [6]:
import json
with open('../all_abstract.json', 'r') as f_:
    jsn = json.load(f_)
print (len(jsn))
counter = 0
# for k,v in jsn.items():
#     if 'free survival' in v.lower():
#         print (v)
#         counter += 1
#     print ('======'*10)
    
#     if counter >= 5:
#         break

3240


In [8]:
import pickle5 as pickle
with open('./Drug_Dev_Processing_DF_citeline.pkl', "rb") as fh:
     to_process = pickle.load(fh)

to_process = to_process#.iloc[:10]

print (to_process.shape)


to_process.head()

(210, 8)


nct_id    other_ids  \
0  NCT00056160  NCT00056160   
1  NCT00057564  NCT00057564   
2  NCT00086580  NCT00086580   
3  NCT00087711  NCT00087711   
4  NCT00090051  NCT00090051   

                                           paragraph  n_arms  \
0  November 22, 2007\n\nN Engl J Med. 2007 Nov 22...       2   
1  March 24, 2008 \n\n J Clin Oncol. 2008 May 1;2...       2   
2  October 11, 2011\n\nThe Lancet Oncology, Early...       2   
3  December 15, 2019 [Meta Analysis]\n\nAnnals of...       2   
4  April 1, 2010 \n\nJournal of Clinical Oncology...       2   

                                           arms_desc  \
0  [Drug: CC-5013 | Drug: Dexamethasone, Drug: De...   
1  [Drug: A (Thalidomide + Dexamethasone), Drug: ...   
2  [Biological: FluCAM [Fludara + Campath], Biolo...   
3  [Drug: pemetrexed | Drug: cisplatin, Drug: gem...   
4  [Drug: Fludarabine Phosphate | Drug: Cyclophos...   

                                      treatment_desc  \
0  [Drug: CC-5013 | Drug: Dexamethasone, Drug: De...   
1  [Drug: A (Thalidomide + Dexamethasone), Drug: ...   
2  [Biological: FluCAM [Fludara + Campath], Biolo...   
3  [Drug: pemetrexed | Drug: cisplatin, Drug: gem...   
4  [Drug: Fludarabine Phosphate | Drug: Cyclophos...   

                          group_types                    nct_id_conditions  
0        [Experimental, Experimental]                     Multiple Myeloma  
1  [Experimental, Placebo Comparator]                     Multiple Myeloma  
2   [Experimental, Active Comparator]  B-Cell Chronic Lymphocytic Leukemia  
3   [Experimental, Active Comparator]        Non Small Cell Lung Carcinoma  
4   [Active Comparator, Experimental]         Chronic Lymphocytic Leukemia

In [9]:
# counter = 0
# to_f = [" Serious Adverse Events "]
# aa_list = []
# for i, row in to_process.iterrows():
#     for tf in to_f:
#         if tf.lower() in row['paragraph'].lower():
#             aa_list.append(tf)
#             print (row['paragraph'])
#             print ('-'*50)
#             if counter == 10:
#                 break
# pd.Series(aa_list).value_counts()


In [10]:
# def normalize_input(txt):
#     txt = txt.replace("(", " ( ").replace(")", " ) ")
#     txt = txt.replace("-", " ")
#     return txt
def normalize_input(txt):
    import re
    txt = txt.replace('·', '.')
    txt = txt.replace("(", " ( ").replace(")", " ) ")
    txt = txt.replace("-", " ")
    txt = txt.replace("vs.", "vs")

    def replace(g):
        return g.group(0).replace(',', ' ').replace(';', ' ').replace(':', ' ')

    txt = re.sub(r'\(.*?\)', replace, txt)
    
    return txt

to_process['paragraph'] =  to_process['paragraph'].apply(normalize_input)
#to_process = list(map(normalize_input, list(jsn.values())))
all_results = p_model.transform(spark.createDataFrame(pd.DataFrame({"text": to_process['paragraph']}))).collect()


In [22]:
NerVisualizer().display(all_results[2], 'ner_merge_chunk')

In [23]:
NerVisualizer().display(all_results[2], 'rex_text_chunk')

In [24]:
RelationExtractionVisualizer().display(all_results[2], relation_col = 'relations', #specify relations column
               document_col = 'document', #specify document column
               show_relations=True, exclude_relations=[])

/opt/conda/lib/python3.8/site-packages/sparknlp_display/relation_extraction.py:366: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  relation_coordinates = np.array(relation_coordinates)


**Post Processing to link entities and prepare output**

In [11]:
import copy

def populate_units(list_vals):
    wo_units = []
    for index, v in enumerate(list_vals):
        v = v.lower()
        found_unit = False
        for un in ['month', 'week', 'day', 'year', '%']:
            if v.find(un) >= 0:
                found_unit = un
                break
        if found_unit != False:
            for a in wo_units:
                wo_units.remove(a)
                list_vals[a] += ' ' + found_unit
        elif 'reached' in v:
            pass
        else:
            wo_units.append(index)
    return list_vals

def process_single_example(res, ntrials, trial_desc, trial_group_type, arms_desc):
    
    all_chunks = {}
    date_chunks = {}
    for r in res['rex_text_chunk']:
        all_chunks[int(r.begin)] = { 'result': r.result, 'entity': r.metadata['entity'], 
                                    'sentence': int(r.metadata['sentence']) }
        if r.metadata['entity'].lower() == 'date' and len(str(r.result).strip()) > 4:
            date_chunks[int(r.begin)] = { 'result': r.result, 'entity': r.metadata['entity'], 
                                    'sentence': int(r.metadata['sentence']) }
    
    sents_list = []
    for r in res['sentence']:
        sents_list.append(r.result)
    
    ## for splitting text by date
#     date_chunks_sents = {}
#     sorted_dates_ents = sorted(list(date_chunks.keys()))
#     st = -1
#     for k in sorted_dates_ents:
#         if st != -1:
#             date_chunks_sents[st] = {'result': '\n'.join(sents_list[date_chunks[st]['sentence']:date_chunks[k]['sentence']])}
#         st = k
        
#     date_chunks_sents[st] = {'result' : '\n'.join(sents_list[date_chunks[st]['sentence']:])}
    
    all_related_eps = {}
    for r in res['relations']:
        if (r.metadata['entity1'].lower() == 'end_point') and (int(r.metadata['entity1_begin']) < int(r.metadata['entity2_begin'])):
            ep = int(r.metadata['entity1_begin'])
            oth = int(r.metadata['entity2_begin'])
        else:
            ep = int(r.metadata['entity2_begin'])
            oth = int(r.metadata['entity1_begin'])
            
        if ep in all_related_eps:
            if oth not in all_related_eps[ep][all_chunks[oth]['entity'].lower()]:
                all_related_eps[ep][all_chunks[oth]['entity'].lower()].append(oth)
        else:
            all_related_eps[ep] = {'ep': ep, 'ep_value':all_chunks[ep]['result'],  
                                   'value': [], 'duration': [], 'hazard_ratio': [], 'final_values': [],
                                  'trial_group': []}
            all_related_eps[ep][all_chunks[oth]['entity'].lower()].append(oth)
    
    def explode_eps(v):
        if v['duration'] != []:
            new_eps = []
            for dur in v['duration']:
                v['new_duration'] = dur
                new_eps.append(v.copy())
            return new_eps
        else:
            v['new_duration'] = None
            return [v]
        
    # divide them by sentence
    by_sent = {}
    for k, v in all_related_eps.items():
        if all_chunks[k]['sentence'] not in by_sent:
            by_sent[all_chunks[k]['sentence']] = explode_eps(v)
        else:
            by_sent[all_chunks[k]['sentence']].extend(explode_eps(v))
    
    #print ('='*100)
    for k, v in by_sent.items():
        dups = []
        removed_dups = []
        for v_ in v:
            nn = v_['ep_value']+all_chunks.get(v_['new_duration'], {'result': ''})['result']
            if nn not in dups:
                removed_dups.append(copy.deepcopy(v_))
            dups.append(nn)
        by_sent[k] = removed_dups
    
            
    for k, v in by_sent.items():
        temp_mapping = {}
        for ind , v_ in enumerate(v):
            if v_['new_duration'] != None: temp_mapping[v_['new_duration'] ] =  ind
            else: temp_mapping[v_['ep']] = ind
        exp_inds = sorted(set(list(temp_mapping.keys())))
        val_inds = sorted(set([ val for v_ in v for val in v_['value']]))
        
        t_rounds = int (len(val_inds) / len(exp_inds))
        #print (t_rounds)
        #print(temp_mapping)
        for tr in range(t_rounds):
            for t_exp_ep in exp_inds:
                frst, smlst, clst, grtst = find_formula(t_exp_ep, val_inds)
                if clst != None:
                    val_inds.remove(clst)
                    v[temp_mapping[t_exp_ep]]['final_values'].append(clst)

    
    
    final_result = []
    for k, v in by_sent.items():
        for v_ in v:
            this_values_mapped = []
            
            this_values_units = [ all_chunks[i]['result'] for i in v_['final_values']]
            this_values_units = populate_units(this_values_units)
            
            for i in this_values_units:
                pp = trial_desc.pop(0)
                ppt = trial_group_type.pop(0)
                #if arms_desc == []: ppa = None else: ppa = arms_desc.pop(0)
                ppa = arms_desc.pop(0)
                this_values_mapped.append( {'value': i, 'trial_group': pp, 'arm_type': ppt, 'arm_desc': ppa})
                trial_desc.append(pp)
                arms_desc.append(ppa)
                trial_group_type.append(ppt)
            
            for tv in this_values_mapped:
                frst, smlst, clst, grtst = find_formula(v_['ep'], list(date_chunks.keys()))
                tv['end_point_date'] = all_chunks.get(smlst, {'result':''})['result']
                #tv['end_point_text_by_date'] = date_chunks_sents.get(smlst, {'result': ''})['result']
                tv['end_point_text'] = sents_list[all_chunks[v_['ep']]['sentence']]
                tv['end_point'] = all_chunks[v_['ep']]['result']
                tv['end_point_type'] = all_chunks.get(v_['new_duration'], {'result':''})['result']
            
            final_result.extend(this_values_mapped)
            
    return pd.DataFrame(final_result)
            

**For this cell, and in order to attach conditions to the drug, `conditions_combined.csv` is required.**

In [12]:
import copy
all_dfs = []
for index, (ind, row) in enumerate(to_process.iterrows()):
    print (row['nct_id'])
    #print (all_results[index])
    #print (row['group_types'])
    this_res = process_single_example(copy.deepcopy(all_results[index]), copy.deepcopy(row['n_arms']), 
                                      copy.deepcopy(row['treatment_desc']), 
                                      copy.deepcopy(row['group_types']), copy.deepcopy(row['arms_desc']))
   
    if this_res.shape[0] > 0:
        this_res['nct_id'] = row['nct_id']
        this_res['other_ids'] = row['other_ids']
        this_res['document'] = row['paragraph']
        this_res['n_arms'] = row['n_arms']
        #this_res['drug_name'] = row['drug_name']
        #this_res['therapy_areas'] = row['therapy_areas']
        this_res['nct_id_condition_names'] = row['nct_id_conditions']
        all_dfs.append(this_res)
to_write = pd.concat(all_dfs, axis=0)
print (to_write.shape)
to_write = to_write.sort_values('end_point')
to_write.head(50)

NCT00056160
NCT00057564
NCT00086580
NCT00087711
NCT00090051
NCT00114101
NCT00179621
NCT00257608
NCT00260832
NCT00281918
NCT00312377
NCT00322452
NCT00330759
NCT00349219
NCT00364351
NCT00373425
NCT00390806
NCT00404924
NCT00405756
NCT00409188
NCT00417911
NCT00418886
NCT00424047
NCT00430365
NCT00443235
NCT00446225
NCT00460317
NCT00461747
NCT00480363
NCT00507416
NCT00540514
NCT00551928
NCT00556712
NCT00644228
NCT00645606
NCT00651261
NCT00656136
NCT00686959
NCT00689936
NCT00722566
NCT00748189
NCT00762034
NCT00773747
NCT00774345
NCT00789373
NCT00805194
NCT00806819
NCT00806923
NCT00822393
NCT00824265
NCT00863746
NCT00874419
NCT00883779
NCT00891384
NCT00932893
NCT00946712
NCT00949650
NCT00961415
NCT00981058
NCT00991211
NCT01010061
NCT01023308
NCT01039376
NCT01040780
NCT01080391
NCT01082549
NCT01085136
NCT01102426
NCT01121393
NCT01168973
NCT01183858
NCT01191801
NCT01239797
NCT01241500
NCT01244191
NCT01282151
NCT01285609
NCT01287741
NCT01302392
NCT01303796
NCT01311687
NCT01313689
NCT01345019
NCT0

value                                        trial_group  \
16   43.6 mos  Drug: Lenalidomide plus low-dose dexamethasone...   
10        10%                                   Drug: ofatumumab   
17        45%  Drug: LoDAC (Low Dose Cytarabine) | Drug: Azac...   
16        20%                                 Drug: gilteritinib   
90        25%  Drug: Lenalidomide | Drug: Prednisone | Drug: ...   
35        40%       Drug: Standard chemotherapy CHOP + Ritiximab   
11        11%  Drug: LoDAC (Low Dose Cytarabine) | Drug: Azac...   
19       6.4%               Drug: Bendamustine | Drug: Rituximab   
20       1.9%                                 Drug: Zanubrutinib   
9         12%                                    Drug: ibrutinib   
7         45%  Drug: LoDAC (Low Dose Cytarabine) | Drug: Azac...   
25        46%               Drug: Thalidomide/Prednisone/Velcade   
24        42%                 Drug: Melphalan/Prednisone/Velcade   
23        24%               Drug: Thalidomide/Prednisone/Velcade   
22        23%                 Drug: Melphalan/Prednisone/Velcade   
21        20%               Drug: Thalidomide/Prednisone/Velcade   
6         20%                                 Drug: gilteritinib   
5         58%  Drug: LoDAC (Low Dose Cytarabine) | Drug: Azac...   
34        51%                                 Drug: Bendamustine   
26        39%                 Drug: Melphalan/Prednisone/Velcade   
8         30%                                   Drug: ofatumumab   
89        20%                                    Drug: Melphalan   
41        22%               Drug: Thalidomide/Prednisone/Velcade   
64        22%                 Drug: Melphalan/Prednisone/Velcade   
10       3.0%            Drug: Dexamethasone | Drug: Carfilzomib   
9       28.8%  Drug: Dexamethasone | Drug: Daratumumab | Drug...   
9       18.8%              Drug: Methotrexate | Drug: Bexarotene   
10         0%                          Drug: Brentuximab Vedotin   
14        89%                          Drug: Brentuximab Vedotin   
54        42%                 Drug: Melphalan/Prednisone/Velcade   
53        25%               Drug: Thalidomide/Prednisone/Velcade   
52        26%                 Drug: Melphalan/Prednisone/Velcade   
15        41%              Drug: Methotrexate | Drug: Bexarotene   
29         4%  Drug: LoDAC (Low Dose Cytarabine) | Drug: Azac...   
49        42%               Drug: Thalidomide/Prednisone/Velcade   
48        25%                 Drug: Melphalan/Prednisone/Velcade   
6       33.8%            Drug: obinutuzumab | Drug: chlorambucil   
20        16%                          Drug: Brentuximab Vedotin   
5       25.7%                                 Drug: chlorambucil   
22         2%                          Drug: Brentuximab Vedotin   
43        36%               Drug: Thalidomide/Prednisone/Velcade   
42        27%                 Drug: Melphalan/Prednisone/Velcade   
4         63%                                 Drug: gilteritinib   
65        27%               Drug: Thalidomide/Prednisone/Velcade   
94  18 months  Drug: Lenalidomide | Drug: Prednisone | Drug: ...   
0       26.9%            Drug: Dexamethasone | Drug: Carfilzomib   
25      33.6%                             Drug: Treatment Choice   
24      17.4%                      Drug: SGI-110 (guadecitabine)   
23      19.4%                             Drug: Treatment Choice   
22      17.4%                      Drug: SGI-110 (guadecitabine)   

             arm_type                                           arm_desc  \
16       Experimental  Drug: Lenalidomide plus low-dose dexamethasone...   
10  Active Comparator                                   Drug: ofatumumab   
17  Active Comparator  Drug: LoDAC (Low Dose Cytarabine) | Drug: Azac...   
16       Experimental                                 Drug: gilteritinib   
90  Active Comparator  Drug: Lenalidomide | Drug: Prednisone | Drug: ...   
35  Active Comparator       Drug: Standard chemotherapy CHOP + Ritiximab   
11  Act

In [13]:
to_write = to_write[['nct_id', 'other_ids', 'document', 'nct_id_condition_names', 'end_point', 'end_point_type', 
          'trial_group', 'value', 'end_point_date', 'end_point_text', 'n_arms', 'arm_type', 'arm_desc']]
to_write['end_point_full'] = to_write['end_point_type'] + ' ' + to_write['end_point']
to_write['end_point_full'] = to_write['end_point_full'].str.strip()
to_write['end_point_full_lower'] = to_write['end_point_full'].str.lower()
to_write.head()


nct_id    other_ids  \
16  NCT00689936  NCT00689936   
10  NCT01578707  NCT01578707   
17  NCT02421939  NCT02421939   
16  NCT02421939  NCT02421939   
90  NCT00551928  NCT00551928   

                                             document  \
16  July 1, 2021\n\nLenalidomide in combination wi...   
10  November 4, 2020 [Pooled analysis]\n\nBlood, V...   
17  January 26, 2022\n\nBlood. 2022 Jan 26;blood.2...   
16  January 26, 2022\n\nBlood. 2022 Jan 26;blood.2...   
90  June 11, 2021 [Pooled analysis]\n\nPresented a...   

                               nct_id_condition_names end_point  \
16                                   Multiple Myeloma        CR   
10  Relapsed or Refractory Chronic Lymphocytic Leu...        CR   
17                      Leukemia, Acute Myeloid (AML)        CR   
16                      Leukemia, Acute Myeloid (AML)        CR   
90        Multiple Myeloma | Newly Diagnosed Patients        CR   

   end_point_type                                        trial_group  \
16                 Drug: Lenalidomide plus low-dose dexamethasone...   
10                                                  Drug: ofatumumab   
17       24 month  Drug: LoDAC (Low Dose Cytarabine) | Drug: Azac...   
16       24 month                                 Drug: gilteritinib   
90                 Drug: Lenalidomide | Drug: Prednisone | Drug: ...   

       value end_point_date  \
16  43.6 mos                  
10       10%                  
17       45%                  
16       20%                  
90       25%                  

                                       end_point_text  n_arms  \
16  Patients who achieved a CR with Rd continuous ...       3   
10  The CR rate  ( CR+CR with incomplete marrow re...       2   
17  Cumulative 24 month relapse rates in gilteriti...       2   
16  Cumulative 24 month relapse rates in gilteriti...       2   
90  > or = VGPR  ( 60% vs 58%  p=0.24 )  and CR  (...       2   

             arm_type                                           arm_desc  \
16       Experimental  Drug: Lenalidomide plus low-dose dexamethasone...   
10  Active Comparator                                   Drug: ofatumumab   
17  Active Comparator  Drug: LoDAC (Low Dose Cytarabine) | Drug: Azac...   
16       Experimental                                 Drug: gilteritinib   
90  Active Comparator  Drug: Lenalidomide | Drug: Prednisone | Drug: ...   

   end_point_full end_point_full_lower  
16             CR                   cr  
10             CR                   cr  
17    24 month CR          24 month cr  
16    24 month CR          24 month cr  
90             CR                   cr

In [14]:
to_write.to_excel('temp.xlsx', index=False)

In [16]:
new_groups = []
for index, group in to_write.groupby(['nct_id', 'end_point_date', 'end_point_full_lower']):
    n_arms = group['n_arms'].values[0]
    
    group = group.iloc[:n_arms]
    new_groups.append(group)

    
result_df = pd.concat(new_groups, axis=0).drop('n_arms', axis=1)
result_df.head()

nct_id    other_ids  \
37  NCT00002663  NCT00002663   
28  NCT00002663  NCT00002663   
35  NCT00002663  NCT00002663   
0   NCT00002663  NCT00002663   
39  NCT00002663  NCT00002663   

                                             document  \
37  Pooled analysis from two phase II trials and o...   
28  Pooled analysis from two phase II trials and o...   
35  Pooled analysis from two phase II trials and o...   
0   Pooled analysis from two phase II trials and o...   
39  Pooled analysis from two phase II trials and o...   

                               nct_id_condition_names          end_point  \
37  EBV-induced Lymphomas | EBV-associated Maligna...  complete response   
28  EBV-induced Lymphomas | EBV-associated Maligna...                 CR   
35  EBV-induced Lymphomas | EBV-associated Maligna...                ORR   
0   EBV-induced Lymphomas | EBV-associated Maligna...                 OS   
39  EBV-induced Lymphomas | EBV-associated Maligna...   overall survival   

   end_point_type                                        trial_group  \
37                 Biological: allogeneic Epstein-Barr virus-spec...   
28                 Biological: allogeneic Epstein-Barr virus-spec...   
35                 Biological: allogeneic Epstein-Barr virus-spec...   
0                  Biological: allogeneic Epstein-Barr virus-spec...   
39                 Biological: allogeneic Epstein-Barr virus-spec...   

             value end_point_date  \
37  12 to 13 month                  
28           42.1%                  
35             69%                  
0              95%                  
39            68 %                  

                                       end_point_text      arm_type  \
37  The median survival for tab cel™ treated patie...  Experimental   
28  An ORR of 63.2%  ( 48/76 )  was observed in al...  Experimental   
35  Tab cel™ was associated with durable ORR  ( CR...  Experimental   
0   The median OS  ( 95% confidence interval [CI] ...  Experimental   
39  One year overall survival for patients with EB...  Experimental   

                                             arm_desc     end_point_full  \
37  Biological: allogeneic Epstein-Barr virus-spec...  complete response   
28  Biological: allogeneic Epstein-Barr virus-spec...                 CR   
35  Biological: allogeneic Epstein-Barr virus-spec...                ORR   
0   Biological: allogeneic Epstein-Barr virus-spec...                 OS   
39  Biological: allogeneic Epstein-Barr virus-spec...   overall survival   

   end_point_full_lower  
37    complete response  
28                   cr  
35                  orr  
0                    os  
39     overall survival

In [17]:
result_df = result_df.drop_duplicates(subset=['nct_id', 'end_point_full_lower', 'end_point_date', 'value'])
result_df = result_df.drop('end_point_full_lower', axis=1)
result_df.head()

nct_id    other_ids  \
37  NCT00002663  NCT00002663   
28  NCT00002663  NCT00002663   
35  NCT00002663  NCT00002663   
0   NCT00002663  NCT00002663   
39  NCT00002663  NCT00002663   

                                             document  \
37  Pooled analysis from two phase II trials and o...   
28  Pooled analysis from two phase II trials and o...   
35  Pooled analysis from two phase II trials and o...   
0   Pooled analysis from two phase II trials and o...   
39  Pooled analysis from two phase II trials and o...   

                               nct_id_condition_names          end_point  \
37  EBV-induced Lymphomas | EBV-associated Maligna...  complete response   
28  EBV-induced Lymphomas | EBV-associated Maligna...                 CR   
35  EBV-induced Lymphomas | EBV-associated Maligna...                ORR   
0   EBV-induced Lymphomas | EBV-associated Maligna...                 OS   
39  EBV-induced Lymphomas | EBV-associated Maligna...   overall survival   

   end_point_type                                        trial_group  \
37                 Biological: allogeneic Epstein-Barr virus-spec...   
28                 Biological: allogeneic Epstein-Barr virus-spec...   
35                 Biological: allogeneic Epstein-Barr virus-spec...   
0                  Biological: allogeneic Epstein-Barr virus-spec...   
39                 Biological: allogeneic Epstein-Barr virus-spec...   

             value end_point_date  \
37  12 to 13 month                  
28           42.1%                  
35             69%                  
0              95%                  
39            68 %                  

                                       end_point_text      arm_type  \
37  The median survival for tab cel™ treated patie...  Experimental   
28  An ORR of 63.2%  ( 48/76 )  was observed in al...  Experimental   
35  Tab cel™ was associated with durable ORR  ( CR...  Experimental   
0   The median OS  ( 95% confidence interval [CI] ...  Experimental   
39  One year overall survival for patients with EB...  Experimental   

                                             arm_desc     end_point_full  
37  Biological: allogeneic Epstein-Barr virus-spec...  complete response  
28  Biological: allogeneic Epstein-Barr virus-spec...                 CR  
35  Biological: allogeneic Epstein-Barr virus-spec...                ORR  
0   Biological: allogeneic Epstein-Barr virus-spec...                 OS  
39  Biological: allogeneic Epstein-Barr virus-spec...   overall survival

In [18]:
result_df.to_excel('drug_dev_results_springer.xlsx', index=False)

In [19]:
c = pd.read_excel('drug_dev_results_springer.xlsx')
c = c.fillna('')
print (c.shape)

(3136, 13)


In [12]:
import pandas as pd
import re
a = pd.read_excel('drug_dev_results_1.xlsx')
b = pd.read_excel('drug_dev_results_2.xlsx')
c = pd.concat([a,b], axis=0)
c = c.fillna('')
print (a.shape, b.shape, c.shape)
#c.to_excel('drug_dev_results_new.xlsx', index=False)

(5901, 13) (5661, 13) (11562, 13)


In [20]:
# def fix_units(value, txt):
#     if 'month' not in value and '%' not in value and 'day' not in value and 'year' not in value and 'week' not in value and 'reached' not in value:
#         if 'median' not in txt.lower():
#             return value+"%"
#         else:
#             return value+' months'
#     return value

# c['value'] = c.apply(lambda x : fix_units(x['value'], x['end_point_type']) , axis=1)
import re
def determine_type(ep):
    ep = ep.lower()
    if 'pfs' in ep or 'progression' in ep:
        return 'PFS'
    elif 'os' in ep or 'overall' in ep:
        return 'OS'
    
    return None

def clean_ep_type(ep, vl):
    if 'median' in ep:
        return 'Median'
    elif 'year' in ep or 'month' in ep or 'day' in ep or 'week' in ep:
        return ep
    elif 'month' in vl:
        return 'Median'
    else:
        return None

c['end_point_normalized'] = c['end_point'].apply(determine_type)
c['end_point_type'] = c.apply(lambda x: clean_ep_type(x['end_point_type'].lower(), x['value'].lower()), axis=1)


In [21]:
normalized_groups_list = []
for index, group in c.groupby(['nct_id', 'end_point_normalized', 'end_point_date', 'end_point_text']):
    unit_found = False
    invalid_group = False
    new_normalized_vals = []
    for v in group['value'].values:
        if 'month' in v.lower(): 
            unit_found = 'month(s)'
            break
        if 'week' in v.lower(): 
            unit_found = 'week(s)'
            break
        if 'day' in v.lower(): 
            unit_found = 'day(s)'
            break
        if 'year' in v.lower(): 
            unit_found = 'year(s)'
            break
        if '%' in v.lower(): 
            unit_found = '%'
            break
            
    for v in group['value'].values:
        v = v.lower().strip()
        if 'reached' in v:
            new_normalized_vals.append('Not Reached')
        elif 'month' not in v and 'day' not in v and 'year' not in v and 'week' not in v and '%' not in v:
            if unit_found != False:
                new_normalized_vals.append(v + ' ' + unit_found)
            else:
                invalid_group = True
        else:
            v = re.sub(r'months?', 'month(s)', v)
            v = re.sub(r'days?', 'day(s)', v)
            v = re.sub(r'years?', 'year(s)', v)
            v = re.sub(r'weeks?', 'week(s)', v)
            if v == 'month(s)' or v == 'day(s)' or v == 'year(s)' or v == 'week(s)' or v == '%':
                new_normalized_vals.append(None)
            else:
                new_normalized_vals.append(v)
        if v == '' or 'patient' in v: 
            invalid_group = True
    
    if invalid_group:
        group['normalized_values'] = None
    else:
        group['normalized_values'] = new_normalized_vals
        
    normalized_groups_list.append(group)
    
c = pd.concat(normalized_groups_list).reset_index(drop=True)


In [22]:

c = c.sort_values(['nct_id', 'end_point_normalized', 'end_point_date', 'end_point_text'])

c = c.dropna(subset=['end_point_type', 'end_point_normalized', 'normalized_values'])

c.head(50)

nct_id    other_ids  \
7   NCT00049517  NCT00049517   
8   NCT00049517  NCT00049517   
9   NCT00056160  NCT00056160   
13  NCT00086580  NCT00086580   
16  NCT00086580  NCT00086580   
17  NCT00086580  NCT00086580   
18  NCT00087711  NCT00087711   
19  NCT00087711  NCT00087711   
20  NCT00087711  NCT00087711   
21  NCT00087711  NCT00087711   
22  NCT00087711  NCT00087711   
23  NCT00090051  NCT00090051   
24  NCT00090051  NCT00090051   
26  NCT00114101  NCT00114101   
30  NCT00179621  NCT00179621   
32  NCT00257608  NCT00257608   
33  NCT00260832  NCT00260832   
38  NCT00281918  NCT00281918   
39  NCT00281918  NCT00281918   
40  NCT00312377  NCT00312377   
41  NCT00312377  NCT00312377   
42  NCT00322452  NCT00322452   
43  NCT00322452  NCT00322452   
45  NCT00330759  NCT00330759   
46  NCT00330759  NCT00330759   
47  NCT00349219  NCT00349219   
48  NCT00349219  NCT00349219   
54  NCT00390793  NCT00390793   
55  NCT00401843  NCT00401843   
56  NCT00401843  NCT00401843   
57  NCT00401843  NCT00401843   
58  NCT00401843  NCT00401843   
63  NCT00403403  NCT00403403   
64  NCT00403403  NCT00403403   
65  NCT00404924  NCT00404924   
66  NCT00404924  NCT00404924   
67  NCT00404924  NCT00404924   
68  NCT00404924  NCT00404924   
71  NCT00405756  NCT00405756   
73  NCT00405756  NCT00405756   
75  NCT00409188  NCT00409188   
76  NCT00409188  NCT00409188   
77  NCT00409188  NCT00409188   
78  NCT00409188  NCT00409188   
81  NCT00415363  NCT00415363   
82  NCT00415363  NCT00415363   
87  NCT00424047  NCT00424047   
89  NCT00430365  NCT00430365   
95  NCT00443235  NCT00443235   
96  NCT00446225  NCT00446225   

                                             document  \
7   High dose  ( 90mg/m2 )  daunorubicin + cytarab...   
8   High dose  ( 90mg/m2 )  daunorubicin + cytarab...   
9   Addition of lenalidomide to high dose dexameth...   
13  Alemtuzumab + fludarabine was associated with ...   
16  Alemtuzumab + fludarabine was associated with ...   
17  Alemtuzumab + fludarabine was associated with ...   
18  First line therapy with pemetrexed + cisplatin...   
19  First line therapy with pemetrexed + cisplatin...   
20  First line therapy with pemetrexed + cisplatin...   
21  First line therapy with pemetrexed + cisplatin...   
22  First line therapy with pemetrexed + cisplatin...   
23  Addition of rituximab to second line therapy w...   
24  Addition of rituximab to second line therapy w...   
26  Maintenance therapy with lenalidomide followin...   
30  Lenalidomide  ( 10 mg/day )  and lenalidomide ...   
32  Adding erlotinib to bevacizumab maintenance di...   
33  Decitabine was most frequently associated with...   
38  Fludarabine + cyclophosphamide was associated ...   
39  Fludarabine + cyclophosphamide was associated ...   
40  Docetaxel + vandetanib had an acceptable toler...   
41  Docetaxel + vandetanib had an acceptable toler...   
42  First line gefitinib had a better tolerability...   
43  First line gefitinib had a better tolerability...   
45  Denosumab and zoledronic acid had similar tole...   
46  Denosumab and zoledronic acid had similar tole...   
47  First line erlotinib then second line cisplati...   
48  First line erlotinib then second line cisplati...   
54  Grade 3 and 4 adverse events in patients with ...   
55  Siltuximab + bortezomib was associated with a ...   
56  Siltuximab + bortezomib was associated with a ...   
57  Siltuximab + bortezomib was associated with a ...   
58  Siltuximab + bortezomib was associated with a ...   
63  Addition of bevacizumab to first line chemothe...   
64  Addition of bevacizumab to first line chemothe...   
65  Vandetanib compared with placebo was associate...   
66  Vandetanib compared with placebo was associate...   
67  Vandetanib compared with placebo was associate...   
68  Vandetanib compared with placebo was associate...   
71  First line lenalidomide [Revlimid] + melphalan...   
73  First line lenalidomide [Revlimid] + melphalan...   
75  Maintenance emepepimut S [Stimuvax,

In [23]:
c['normalized_values'] = c.apply(lambda row: None if row['end_point_type'] == 'Median' and '%' in row['normalized_values'] else row['normalized_values'], axis=1)

c = c.dropna(subset=['end_point_type', 'end_point_normalized', 'normalized_values'])


In [24]:
def normalize_ep_type(v):
    v = v.lower()
    if 'median' in v:
        return 'Median'
    type_ = None
    if 'year' in v:
        type_ = 'year'
        v = re.sub(r'years?' , '', v).strip()
    elif 'week' in v:
        type_ = 'week'
        v = re.sub(r'weeks?' , '', v).strip()
    elif 'month' in v:
        type_ = 'month'
        v = re.sub(r'months?' , '', v).strip()
    elif 'day' in v:
        type_ = 'day'
        v = re.sub(r'days?' , '', v).strip()
    v = re.sub(r'one' , '1', v)
    v = re.sub(r'two', '2', v)
    v = re.sub(r'three', '3', v)
    v = re.sub(r'four', '4', v)
    v = re.sub(r'five', '5', v)
    v = re.sub(r'six', '6', v)
    v = re.sub(r'seven', '7', v)
    v = re.sub(r'eight', '8', v)
    v = re.sub(r'nine', '9', v)
    v = re.sub(r'ten', '10', v)
    v = re.sub(r'eleven', '11', v)
    v = re.sub(r'twelve', '12', v)
    print (v, type_)
    if v.strip() == '':
        return None
    return v + ' ' + type_

c['end_point_type'] = c['end_point_type'].apply(normalize_ep_type)


In [25]:
c = c.dropna(subset=['end_point_type', 'end_point_normalized', 'normalized_values'])


In [26]:
bad_ncts = [ a['nct_id'] for i, a in c.iterrows() if not a['normalized_values'][0].isdigit() and a['normalized_values'] != 'Not Reached' ]
c = c[~c['nct_id'].isin(bad_ncts)]
c.head(50)


nct_id    other_ids  \
7   NCT00049517  NCT00049517   
8   NCT00049517  NCT00049517   
9   NCT00056160  NCT00056160   
13  NCT00086580  NCT00086580   
16  NCT00086580  NCT00086580   
17  NCT00086580  NCT00086580   
18  NCT00087711  NCT00087711   
19  NCT00087711  NCT00087711   
20  NCT00087711  NCT00087711   
21  NCT00087711  NCT00087711   
22  NCT00087711  NCT00087711   
23  NCT00090051  NCT00090051   
24  NCT00090051  NCT00090051   
26  NCT00114101  NCT00114101   
30  NCT00179621  NCT00179621   
32  NCT00257608  NCT00257608   
33  NCT00260832  NCT00260832   
38  NCT00281918  NCT00281918   
39  NCT00281918  NCT00281918   
40  NCT00312377  NCT00312377   
41  NCT00312377  NCT00312377   
42  NCT00322452  NCT00322452   
43  NCT00322452  NCT00322452   
45  NCT00330759  NCT00330759   
46  NCT00330759  NCT00330759   
47  NCT00349219  NCT00349219   
48  NCT00349219  NCT00349219   
54  NCT00390793  NCT00390793   
55  NCT00401843  NCT00401843   
56  NCT00401843  NCT00401843   
57  NCT00401843  NCT00401843   
58  NCT00401843  NCT00401843   
63  NCT00403403  NCT00403403   
64  NCT00403403  NCT00403403   
65  NCT00404924  NCT00404924   
66  NCT00404924  NCT00404924   
67  NCT00404924  NCT00404924   
68  NCT00404924  NCT00404924   
71  NCT00405756  NCT00405756   
73  NCT00405756  NCT00405756   
75  NCT00409188  NCT00409188   
76  NCT00409188  NCT00409188   
77  NCT00409188  NCT00409188   
78  NCT00409188  NCT00409188   
81  NCT00415363  NCT00415363   
82  NCT00415363  NCT00415363   
87  NCT00424047  NCT00424047   
89  NCT00430365  NCT00430365   
95  NCT00443235  NCT00443235   
96  NCT00446225  NCT00446225   

                                             document  \
7   High dose  ( 90mg/m2 )  daunorubicin + cytarab...   
8   High dose  ( 90mg/m2 )  daunorubicin + cytarab...   
9   Addition of lenalidomide to high dose dexameth...   
13  Alemtuzumab + fludarabine was associated with ...   
16  Alemtuzumab + fludarabine was associated with ...   
17  Alemtuzumab + fludarabine was associated with ...   
18  First line therapy with pemetrexed + cisplatin...   
19  First line therapy with pemetrexed + cisplatin...   
20  First line therapy with pemetrexed + cisplatin...   
21  First line therapy with pemetrexed + cisplatin...   
22  First line therapy with pemetrexed + cisplatin...   
23  Addition of rituximab to second line therapy w...   
24  Addition of rituximab to second line therapy w...   
26  Maintenance therapy with lenalidomide followin...   
30  Lenalidomide  ( 10 mg/day )  and lenalidomide ...   
32  Adding erlotinib to bevacizumab maintenance di...   
33  Decitabine was most frequently associated with...   
38  Fludarabine + cyclophosphamide was associated ...   
39  Fludarabine + cyclophosphamide was associated ...   
40  Docetaxel + vandetanib had an acceptable toler...   
41  Docetaxel + vandetanib had an acceptable toler...   
42  First line gefitinib had a better tolerability...   
43  First line gefitinib had a better tolerability...   
45  Denosumab and zoledronic acid had similar tole...   
46  Denosumab and zoledronic acid had similar tole...   
47  First line erlotinib then second line cisplati...   
48  First line erlotinib then second line cisplati...   
54  Grade 3 and 4 adverse events in patients with ...   
55  Siltuximab + bortezomib was associated with a ...   
56  Siltuximab + bortezomib was associated with a ...   
57  Siltuximab + bortezomib was associated with a ...   
58  Siltuximab + bortezomib was associated with a ...   
63  Addition of bevacizumab to first line chemothe...   
64  Addition of bevacizumab to first line chemothe...   
65  Vandetanib compared with placebo was associate...   
66  Vandetanib compared with placebo was associate...   
67  Vandetanib compared with placebo was associate...   
68  Vandetanib compared with placebo was associate...   
71  First line lenalidomide [Revlimid] + melphalan...   
73  First line lenalidomide [Revlimid] + melphalan...   
75  Maintenance emepepimut S [Stimuvax,

In [27]:

c.to_excel('drug_dev_results_springer.xlsx', index=False)

In [28]:
import pandas as pd
a = pd.read_excel('drug_dev_results_citeline.xlsx')
b = pd.read_excel('drug_dev_results_springer.xlsx')
c = pd.read_excel('drug_dev_results.xlsx')

a, b, c = map(set, (a['nct_id'].values,b['nct_id'].values,c['nct_id'].values))                                                                                                                                                                                                                                                                                          

a.intersection(b, c)  

{'NCT00056160',
 'NCT00086580',
 'NCT00087711',
 'NCT00090051',
 'NCT00257608',
 'NCT00260832',
 'NCT00281918',
 'NCT00312377',
 'NCT00403403',
 'NCT00404924',
 'NCT00405756',
 'NCT00409188',
 'NCT00430365',
 'NCT00443235',
 'NCT00446225',
 'NCT00460317',
 'NCT00507416',
 'NCT00517530',
 'NCT00527735',
 'NCT00540514',
 'NCT00551928',
 'NCT00590187',
 'NCT00633789',
 'NCT00644228',
 'NCT00656136',
 'NCT00689936',
 'NCT00700180',
 'NCT00716534',
 'NCT00742560',
 'NCT00748189',
 'NCT00768755',
 'NCT00774345',
 'NCT00777036',
 'NCT00789373',
 'NCT00805194',
 'NCT00806819',
 'NCT00824408',
 'NCT00833833',
 'NCT00854308',
 'NCT00874419',
 'NCT00883779',
 'NCT00890825',
 'NCT00932893',
 'NCT00940225',
 'NCT00949650',
 'NCT00961415',
 'NCT00981058',
 'NCT00991211',
 'NCT00994097',
 'NCT01023308',
 'NCT01040780',
 'NCT01045421',
 'NCT01053949',
 'NCT01080391',
 'NCT01083602',
 'NCT01088048',
 'NCT01102426',
 'NCT01121393',
 'NCT01138163',
 'NCT01160744',
 'NCT01168973',
 'NCT01191801',
 'NCT012